In [164]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [165]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text =f.read()
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)
vocab_size


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


81

In [166]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l:' '.join([int_to_string[i] for i in l])

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])
# encoded_hello=encode('hello')
# decoded_hello=decode(encoded_hello)
# print(decoded_hello)

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [167]:
# n=int(0.8*len(data))
# train_data=data[:n]
# val_data=data[n:]

In [168]:
# block_size=8
# # basically we look at how far is the target from prediction thus we reduce that error with optimizer
# x=train_data[:block_size]
# y=train_data[1:block_size+1]

# for t in range(block_size):
#     context=x[:t+1]
#     target=y[t]
#     print('when input is ',context,'target is ',target)

In [195]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
## get the batch sixe times sequence of words of given block size of 
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix take some random interger between the data size-block_zixe and of the lenght batch_size which is 4 here
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) # so this will be processed in cuda gpu as we have intailised before
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[71, 58,  1, 57, 68, 79, 58, 67],
        [56, 64, 67, 68, 76, 65, 58, 57],
        [ 1, 72, 66, 68, 68, 73, 61, 62],
        [54, 72, 68, 65,  1, 54, 67, 57]], device='cuda:0')
targets:
tensor([[58,  1, 57, 68, 79, 58, 67, 72],
        [64, 67, 68, 76, 65, 58, 57, 60],
        [72, 66, 68, 68, 73, 61, 62, 67],
        [72, 68, 65,  1, 54, 67, 57,  1]], device='cuda:0')


In [170]:
# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

In [171]:
block_size=8
# basically we look at how far is the target from prediction thus we reduce that error with optimizer
x=train_data[:block_size]
y=train_data[1:block_size+1]
# this what bigram is it will predict the next character based on the present sequence of character 
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print('when input is ',context,'target is ',target)

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)


In [172]:
import torch.nn as nn
from torch.nn import functional as F

In [196]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        # define losses  zeros of lenght max_iter
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split) # get the
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [174]:
class BigramLanguageModel(nn.Module): # what it does is if we use any nn.Module like nn.Linear then it same thing as it will make the parameter learnable by nn.module
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)

        if targets is None:
            loss=None
        else:
            # B=bath_size T= time , C=Channel(vocab size)
            B,T,C=logits.shape # we basically unpacking the tensor to pack them in size as we want by using view function
            logits=logits.view(B*T,C) # so here wee are making B,T a single parameter and C as second
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        
        
        return logits,loss

    def generate(self,index,max_new_tokens):
        #indexing is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the next word prediction for bigram
            logits,loss=self.forward(index)
            # last word is our only focus as this is bigram model
            logits=logits[:,-1,:]  # (B,C)
            # get the prob by softmax
            probs=F.softmax(logits,dim=-1) #(B,C)
            # next index take this from the ditribution
            index_next=torch.multinomial(probs,num_samples=1) #(B,1) get one sample
            # cancatenate the word in the running sequene 
            index=torch.cat((index,index_next),dim=1) #(B,T+1)
        return index   

model=BigramLanguageModel(vocab_size)
m=model.to(device) # give it gpu

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)





    


 . Q : , 5 O A q k ] ' ! y V t y - Q o C - * V V U t b ] 
 o ?   R L M Q ! n - i G . ' ' A j ! - _ e N " w " U * ] V U D U e S   s 6 ' O Y g N a I ( * O V ( U ' & k 3 I q ( 9 e u 5   b g q ﻿ 8 Y ? ' w l F m F f n ) f _ D N 4 E 9 " g R Y G ' F 2 0 m 
 K J B U N _ h Y ? 7 D A : N n w O W K F W H 7 V ] R b h 2 B ! O s [ N 7 1 7 G A U D G m ﻿ p f b F Z K O Y ? _ 3 R Y T S ﻿ M k n B e e 8 J y - m R Y Q B ! P S x r ? p 1 I ? j G x r q : 7 M R _ 4 k f 9 m D c 7 . u S * ﻿ w J 5   T h s D O y h T - e _ _ y n 1 V E I e G 
 K m B H V U q 9 g V Z p ' 2 k K e R V ( j f ? [ O - C d j j d z ] Z r 2 
 U L R Y a w 8 R J & S I z B i 5 6 6 r q : p m 3 T & k f m p Q 8 W   , ; c v s " * I _ 4 Q p v ? k   H Y - b J 3 t V q Q 0 x c v I ( 9 I _ p q F k X w o S ﻿ 
 8 y - . 7 T 6 : w [ q g p s ﻿ q Y ? n c r L R 8 , 
 1 9 m P Z _ K L Z d ? c U : ; n 3 1 1 T w S W X 
 K x : b 0 ) D u y * Q M 0   v ; 6 K 0 u J 3 d   7 y K 8 K 9 C w r L ' * 9 I U : ' _ D O L k a M c b R _ g R L Y g T a ! j d M [ w " D ? s Z B : ﻿ 

In [190]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.919, val loss: 2.891
step: 250, train loss: 2.860, val loss: 2.914
step: 500, train loss: 2.833, val loss: 2.889
step: 750, train loss: 2.836, val loss: 2.871
2.4867420196533203


In [192]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 q ﻿ , 6 r e d & X n g .   a n 
 s , 
 C h i m p & & & 4 g - I ' m y   m y ) ? ] V l m   y ,   t r O V q q r e   t o r e b s P u S & . B y g a n o i o k N e e d   v ﻿ - l E Q ; N I 7 U 6 K ﻿ r e w h T F G o f r o   m * m , 0 N * R I   t e n v 8 Y I ﻿ 8 D R Y ? , 
 l l   ( e   a c k l d 
 t e o 5 ] s 
   w o n i e c   a 
 p p o f 9 k e n d   S   o   d   e m S ﻿ 
 n   I a k [ p l i b e _ r   p   d   T h e   l s h a i l   c l i s f   e d 
 " X ] c t h y 
 D u c o   o   c " L e l _ h e r d   w o l Y   t , p s ?   * Z Z H   y ( 4 9 Y A J e d ; C d   u W 7 W i F f a ! f y , z a t a r . . & 4 "   d l f   s e   c M Q D o s h   t i Q w n E Z q E U , E U E ' p e f   T 0 q q & Y Z d   d ' K P V ; d a n U k I l ! [ K 4 c l O a k a k e n w e n 
 s h e x   t h 4 C W S ' F R J f h a u e   f a r e d   b e , 
 t h   l y * Q " q D ? 3 ; B : p q f e v r e ] C r i f y , L ( E T w s l   m ,   g s e r e R 8 4 F A a T a n e   d 0 ﻿ Q ( J Y ? H O & F ] j Z r e l   o 
 f y u ' 3 C a g q q 7 [ Z   y   i z h e 

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim